### 3. Sa se scrie un program care primeste la intrare elementele unei expresii regulate (alfabetul expresiei, expresia propriu -zisa (in forma prefixata sau infixata - adica forma naturala), care contine 3 tipuri de operatori: reuniune, concatenare si iteratie Kleene (*)). Sa se determine un automat finit nedeterminist cu lambda-tranzitii (sa se afiseze elementele sale) care recunoaste acelasi limbaj ca cel descris de expresia regulata. Programul afiseaza si graful care corespunde noului automat (grafic 1 punct).

In [1]:
def higherPrecedence(a, b):
    p = ["|", ".", "*"]
    return p.index(a) > p.index(b)

def postfix(regexp):
    regexp = regexp.replace(" ", "") # elimin posibilele spatii din input
    
    # adaug punct "." intre simboluri consecutive
    temp = []
    for i in range(len(regexp)):
        if i != 0\
            and (regexp[i-1] in alphabet or regexp[i-1] == ")" or regexp[i-1] == "*")\
            and (regexp[i]  in alphabet or regexp[i] == "("):
            temp.append(".")
        temp.append(regexp[i])
    regexp = temp
    
    # shunting yard algorithm
    stack = []
    output = ""

    for c in regexp:
        if c in alphabet:
            output = output + c
            continue
        if c == ")":
            while len(stack) != 0 and stack[-1] != "(":
                output = output + stack.pop()
            stack.pop()
        elif c == "(":
            stack.append(c)
        elif c == "*":
            output = output + c
        elif len(stack) == 0 or stack[-1] == "(" or higherPrecedence(c, stack[-1]):
            stack.append(c)
        else:
            while len(stack) != 0 and stack[-1] != "(" and not higherPrecedence(c, stack[-1]):
                output = output + stack.pop()
            stack.append(c)

    while len(stack) != 0:
        output = output + stack.pop()

    return output

alphabet = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + \
           [chr(i) for i in range(ord('a'), ord('z') + 1)] + \
           [chr(i) for i in range(ord('0'), ord('9') + 1)] + \
           ['@']

In [2]:
 # FORMA INFIXATA -> FORMA POSTFIXATA
expressions = ['(0*10*)', 'a(a|b)*b', '(a|b)*abb', '(a|b)*abb(a|b)*']
for e in expressions:
    pr = postfix(e)
    print(e + " ====== " + pr)

(0*10*) ====== 0*1.0*.
a(a|b)*b ====== aab|*.b.
(a|b)*abb ====== ab|*a.b.b.
(a|b)*abb(a|b)* ====== ab|*a.b.b.ab|*.


In [4]:
SYMBOL = 1
CONCAT = 2
UNION  = 3
KLEENE = 4

class Tree:
    def __init__(self, _type, value):
        self._type = _type # symbol, concat, union, kleene
        self.value = value # caracter din alfabet SAU | . * 
        self.left = None
        self.right = None

def constructTree(regex):
    stack = []
    for c in regex:
        if c in alphabet:
            stack.append(Tree(SYMBOL, c))
        else:
            if c == "|":  # REUNIUNE
                z = Tree(UNION, '|')
                z.right = stack.pop()
                z.left = stack.pop()
            elif c == ".":  # CONCATENARE
                z = Tree(CONCAT, '.')
                z.right = stack.pop()
                z.left = stack.pop()
            elif c == "*":  # ITERATIE KLEENE
                z = Tree(KLEENE, '*')
                z.left = stack.pop()
            stack.append(z)

    return stack[0]

# def inorder(et):
#     if et._type == SYMBOL:
#         print(et.value)
#     elif et._type == CONCAT:
#         inorder(et.left)
#         print(et.value)
#         inorder(et.right)
#     elif et._type == UNION:
#         inorder(et.left)
#         print(et.value)
#         inorder(et.right)
#     elif et._type == KLEENE:
#         inorder(et.left)
#         print(et.value)

In [30]:
# for e in expressions:
#     pr = postfix(e) # pr = 0*1.0*.
#     et = constructTree(pr)
#     inorder(et)
#     break

In [11]:
## DE INTRODUS EXPRESIA REGEX IN FORMA INFIXATA 
pr = postfix('(0*10*)') # calculez forma postfixata
pr = postfix('(00|11)*01*')
et = constructTree(pr) # construiesc arborele
# inorder(et)

In [12]:
class FiniteAutomataState:
    def __init__(self):
        self.next_state = {}

def evalRegex(et):
    # calculeaza AFN cu lambda tranzitii din arborele dat(care este regex)
    if et._type == SYMBOL:
        return evalRegexSymbol(et)
    elif et._type == CONCAT:
        return evalRegexConcat(et)
    elif et._type == UNION:
        return evalRegexUnion(et)
    elif et._type == KLEENE:
        return evalRegexKleene(et)

def evalRegexSymbol(et):
    start_state = FiniteAutomataState()
    end_state   = FiniteAutomataState()
    
    start_state.next_state[et.value] = [end_state] # tranzitie cu simbolul de la f1 la s2 
    
    return start_state, end_state

def evalRegexConcat(et):
    left_nfa  = evalRegex(et.left) # automatul 1
    right_nfa = evalRegex(et.right) # automatul 2

    left_nfa[1].next_state['lambda'] = [right_nfa[0]]  # tranzitie lambda de la f1 la s2 
    return left_nfa[0], right_nfa[1] # s1 este starea initiala, f2 este starea finala

def evalRegexUnion(et):
    start_state = FiniteAutomataState()  # stare de start s
    end_state   = FiniteAutomataState() # stare finala f

    up_nfa   = evalRegex(et.left) # automatul 1 cu s1 f1 stari 
    down_nfa = evalRegex(et.right) # automatul 2 cu s2 f2 stari

    start_state.next_state['lambda'] = [up_nfa[0], down_nfa[0]] # tranzitii lambda de la s la s1 si s2
    up_nfa[1].next_state['lambda'] = [end_state] # tranzitie lambda de la f1 la f
    down_nfa[1].next_state['lambda'] = [end_state] # tranzitie lambda de la f2 la f

    return start_state, end_state

def evalRegexKleene(et):
    start_state = FiniteAutomataState()  # stare de start s
    end_state   = FiniteAutomataState() # stare finala f

    sub_nfa = evalRegex(et.left) # automatul initial

    start_state.next_state['lambda'] = [sub_nfa[0], end_state] # tranzitii lambda de la s la s1 si f
    sub_nfa[1].next_state['lambda'] = [sub_nfa[0], end_state]  # tranzitii lambda de la f1 la s1 si f

    return start_state, end_state


def printStateTransitions(state, states_done, symbol_table, f = None, f2 = None):
    if state in states_done:
        return
    states_done.append(state)
    for symbol in list(state.next_state):
        line_output = "q" + str(symbol_table[state]) + "\t\t" + symbol + "\t\t\t" # ex: q0    epsilon   
        file_output = str(symbol_table[state]) + " " + symbol + " "
        
        if f is not None:
            f.write(str(symbol_table[state]) + '\n') ## indicele starilor din care ies arce
        
        # for next states (MAX 2)
        aux = [] 
        for ns in state.next_state[symbol]:
            if ns not in symbol_table:
                symbol_table[ns] = 1 + sorted(symbol_table.values())[-1]
            line_output = line_output + "q" + str(symbol_table[ns]) + " " # qi qj
            
            aux.append(str(symbol_table[ns])) # retin starile in care intra arcele 
            
        if f2 is not None:
            for item in aux:
                f2.write(file_output + item + "\n")
        if f:
            print(line_output)
        
        for ns in state.next_state[symbol]:
            printStateTransitions(ns, states_done, symbol_table, f, f2)
    
def printTransitionTable(finite_automata, f = None, f2 = None):
    if f:
        print("State\t\tSymbol\t\t\tNext state")
    printStateTransitions(finite_automata[0], [], {finite_automata[0]:0}, f, f2)

In [13]:
fa = evalRegex(et)

fileName = "stari_cu_arce.txt" 
with open(fileName, "w") as f:
    printTransitionTable(fa, f, f2 = None)

State		Symbol			Next state
q0		lambda			q1 q2 
q1		lambda			q3 q4 
q3		0			q5 
q5		lambda			q6 
q6		0			q7 
q7		lambda			q8 
q8		lambda			q1 q2 
q2		lambda			q9 
q9		0			q10 
q10		lambda			q11 
q11		lambda			q12 q13 
q12		1			q14 
q14		lambda			q12 q13 
q4		1			q15 
q15		lambda			q16 
q16		1			q17 
q17		lambda			q8 


In [14]:
# caut starea finala (cea din care nu mai ies arce)
l = []
with open(fileName, "r") as f:
    lines = f.readlines()
    for line in lines:
        l.append(int(line.strip()))

final_state = -1
for elem in range(len(l)+1):
    if elem not in l:
        final_state = elem
print('final state:', 'q' + str(final_state))

final state: q13


In [15]:
# scriu tranzitiile
fileName = "tranzitii.txt" 
with open(fileName, "w") as f2:
    printTransitionTable(fa, f2 = f2) 

In [16]:
# afisare 
# tranzitii = []
# with open(fileName, "r") as file:
#     lines = file.readlines()
#     for line in lines:
#         print(line.strip().split())

In [17]:
from graphviz import Digraph, render

fa = Digraph('nfa', 'nondeterministic_finite_state_machine', format = 'png')
fa.attr(rankdir='LR') # directia grafului (de la stg la dr)

# starea finala
fa.attr('node', shape = 'doublecircle')
fa.node('q' + str(final_state))

fa.attr('node', shape = 'circle')
with open(fileName, "r") as file:
    lines = file.readlines()
    for line in lines:
        row = line.strip().split()
        fa.edge('q' + row[0], 'q' + row[2], label = row[1])

# starea initiala
fa.attr('node', shape = 'point')
fa.edge('', 'q0')
fa.view()

del fa